In [339]:
using Random
using DataFrames
using CSV
using Statistics

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 400
const MXV = 40
const MNV = 2

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

fpart (generic function with 1 method)

In [340]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [341]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [354]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [355]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    while budget_sofar > budget
        hf_ratio = rand()
        l_ratio = rand()
        h_temp = rand() + 0.1
        f_temp = rand() + 0.1
        budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter("No Name [Scheme A]", h, f, l, t)
end



random_scheme_a (generic function with 6 methods)

In [356]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0


for i in 1:1000
    hf_ratio = rand()
    l_ratio = 0.25 + 0.5 * rand()
    h_temp = rand() + 0.1
    f_temp = rand() + 0.1
    budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
    h_ratio = h_temp/sqrt(h_temp * f_temp)
    f_ratio = f_temp/sqrt(h_temp * f_temp)
    h = max(h0, round(sqrt(budget_hf) * h_ratio))
    f = max(f0, round(sqrt(budget_hf) * f_ratio))
    budget_hf = h * f
    budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
    lraw = l0 + ( budget_remain * l_ratio )/h
    traw = t0 + ( budget_remain * (1 - l_ratio) )/f
    l = trunc(lraw)
    t = trunc(traw)
    budget_sofar = h * f + h * l + f * t
    if (budget - budget_sofar) >= min(h, f)
        could_add_l = (budget - budget_sofar) >= h
        could_add_t = (budget - budget_sofar) >= f
        if could_add_l && !could_add_t
            l = l+trunc((budget - budget_sofar)/h)
        end
        if !could_add_l && could_add_t
            t = t+trunc((budget - budget_sofar)/f)
        end
        if could_add_l && could_add_t
            if fpart(lraw) > fpart(traw)
                l = l+trunc((budget - budget_sofar)/h)
            else
                t = t+trunc((budget - budget_sofar)/f)
            end
        end
    end

    budget_sofar = h * f + h * l + f * t
    if (budget - budget_sofar) >= min(h, f)
        hf_ratio0 = hf_ratio
        l_ratio0 = l_ratio
        h_temp0 = h_temp
        f_temp0 = f_temp
        println((h, f, l, t, budget_hf, budget_sofar))
    end
end

In [357]:
hf_ratio = hf_ratio0
l_ratio = l_ratio0
h_temp = h_temp0
f_temp = f_temp0

0

In [358]:
budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
lraw = l0 + ( budget_remain * l_ratio )/h
traw = t0 + ( budget_remain * (1 - l_ratio) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
println((h, f, l, t, budget_hf, budget_sofar))

(NaN, NaN, NaN, NaN, NaN, NaN)


In [359]:
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+1
    end
    if !could_add_l && could_add_t
        t = t+1
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+1
        else
            t = t+1
        end
    end
end

budget_sofar = h * f + h * l + f * t
println((h, f, l, t, budget_hf, budget_sofar))

(NaN, NaN, NaN, NaN, NaN, NaN)


## Testing Random Scheme A

In [360]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("No Name [Scheme A]", 7, 25, 10, 6)

395

In [361]:
rt = random_team(() -> rand_rename(random_scheme_a()), 100)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Kashtyn the Blimp,26,7,7,5
2,Maelia the Notebook,10,16,11,8
3,Super Duck,14,8,7,23
4,Fiery Duck,15,13,3,12
5,Pug Maryjean,11,20,9,4
6,Magnificent Urchin,4,29,20,7
7,Nice Scorpion,11,18,10,5
8,Disciplined Yak,12,3,22,33
9,Friendly Jaelyn,5,9,51,11


In [362]:
counters = random_team(() -> pick_best_rdmly(library, rt, 9), 50)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Xpanded Airplane,16,10,8,11
2,Leopard Jathniel,10,11,7,20
3,Dynamic Loretta,11,11,12,13
4,Gallant Pizza,8,15,10,13
5,Hoor the Dynamic,14,10,11,10
6,Kitten Cylia,17,7,12,11
7,Boot Rainy,11,16,2,12
8,Maiara the Cube,13,10,10,14
9,Kangaroo Bo,25,11,4,2


In [363]:
for f in rt
    v = eval_battle_list(f, counters)
    if v < -35
        println(f)
        println(v)
    end
end

Fighter("Maelia the Notebook", 10, 16, 11, 8)
-36
Fighter("Magnificent Urchin", 4, 29, 20, 7)
-36
Fighter("Friendly Jaelyn", 5, 9, 51, 11)
-46
Fighter("Powerful Martini", 8, 5, 24, 33)
-36
Fighter("Lorraina the Heroic", 6, 29, 3, 7)
-42
Fighter("Dangerous Nayleah", 6, 5, 15, 56)
-40
Fighter("Ahnika the Good", 5, 13, 56, 4)
-50
Fighter("Sassy Robot", 6, 27, 8, 7)
-42
Fighter("Joker Aramis", 7, 6, 30, 24)
-40
Fighter("Iguana Ailyn", 6, 28, 20, 4)
-46
Fighter("Olivier the Xtraverted", 6, 23, 13, 8)
-36
Fighter("Astou the Noble", 7, 6, 39, 14)
-46
Fighter("Dearri the Frog", 10, 22, 9, 4)
-36
Fighter("Karston the Koala", 10, 8, 29, 3)
-36
Fighter("Good Khora", 6, 8, 46, 9)
-50
Fighter("Commander Dafne", 10, 18, 11, 6)
-38
Fighter("Hadasa the Jackpot", 8, 7, 40, 3)
-46
Fighter("Balanced Davone", 10, 17, 16, 4)
-36
Fighter("Adel the Iguana", 7, 27, 7, 6)
-42
Fighter("Magnificent Quilt", 5, 25, 25, 6)
-42
Fighter("Likable Ninja", 9, 18, 18, 4)
-38
Fighter("Colin the Diamond", 5, 6, 60, 11)
-50

In [364]:
for f in rt
    v = eval_battle_list(f, counters)
    if v > 10
        println(f)
        println(v)
    end
end

Fighter("Generous Mosquito", 18, 9, 12, 2)
26
Fighter("Flirtatious Mylani", 19, 10, 9, 3)
16
Fighter("True the Rogue", 20, 10, 9, 2)
24
Fighter("Rahel the Triangle", 6, 15, 9, 17)
18
Fighter("Shy Mango", 21, 9, 9, 2)
22
Fighter("Wise Reesa", 17, 14, 6, 4)
16
Fighter("Dynamic Mushroom", 17, 6, 12, 15)
28
Fighter("Willful Violet", 20, 9, 9, 4)
22
Fighter("Dedrick the Mosquito", 17, 15, 6, 2)
18


In [396]:
evaluate_generator(random_fighter, 100, 20)

-0.5465

In [397]:
evaluate_generator(random_scheme_a, 100, 20)

-0.5635

In [398]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.53

In [399]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.504

In [401]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.48850000000000016

In [402]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.5385000000000001